In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cudf
pd.set_option('display.max_columns', 500)

# Standard plotly imports
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
colorMap = sns.light_palette("blue", as_cmap=True)

import gc
import cufflinks
import cufflinks as cf
import plotly.figure_factory as ff
import os

import warnings   #控制警告訊息
warnings.filterwarnings("ignore")


In [2]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set


import xgboost as xgb
print("XGBoost version:", xgb.__version__)

XGBoost version: 1.2.1


ReadData

In [3]:
%%time
train_cudf  = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train = train_cudf.to_pandas()
del train_cudf
features = pd.read_csv('../input/jane-street-market-prediction/features.csv')
# example_test = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
# sample_prediction_df = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')
print ("Data is loaded!")

Data is loaded!
CPU times: user 9.26 s, sys: 5.64 s, total: 14.9 s
Wall time: 1min 18s


Data normalized

In [4]:
#尋找missvalues
missvalues = train['feature_7'].isnull().sum()
allvalues = np.product(train['feature_7'].shape)
print(missvalues / allvalues * 100)

16.445784568944205


In [5]:
train = train[train['weight'] != 0]  # weight == 0 對訓練沒有幫助

train['action'] = (train['resp'].values > 0).astype('int') #製作label resp(回報)>0時投資，等於或小於時不投資

y = train['action']
x = train.drop(['date','weight','resp_1','resp_2','resp_3','resp_4','resp','action','ts_id'],axis = 1)
#x = x.fillna(x.mean())

#print(x)
#print(y)


將7、8、72、78以眾數填補缺失值
再將其他用平均值填補缺失值

In [6]:
col = {'feature_7': np.product(x['feature_7'].mode()),'feature_8': np.product(x['feature_8'].mode()),'feature_72': np.product(x['feature_72'].mode()),'feature_78': np.product(x['feature_78'].mode())}
x = x.fillna(value=col)
x = x.fillna(x.mean())
print(x)

         feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
1               -1  -1.349537  -1.704709   0.068058   0.028432   0.193794   
4                1  -3.172026  -3.093182  -0.161518  -0.128149  -0.195006   
6               -1  -3.172026  -3.093182  -0.030588  -0.043175   0.097058   
7               -1   0.446050  -0.466210   0.498751   0.244116   0.412528   
8                1  -3.172026  -3.093182  -0.363836  -0.291496   0.128422   
...            ...        ...        ...        ...        ...        ...   
2390444         -1   1.538675   2.530447   2.494852   3.263345   1.613620   
2390446          1   0.270380  -1.231874  -5.802676  -3.172423  -4.357278   
2390478         -1  -0.134380   0.160580   1.292513   1.453954   0.605912   
2390481         -1  -0.779554  -0.597258   0.674234   0.735692  -0.153732   
2390489         -1  -1.463757  -1.107228  -2.286985  -3.156451  -1.690676   

         feature_6  feature_7  feature_8  feature_9  feature_10  feature_11

試用Tag取得有分類資料進行比對

In [7]:
# fig, ax = plt.subplots(figsize=(20, 10))

# tag = np.copy(np.array(features["tag_0"])) == True  #取得tag_0並獲得tag分類一起的feature位置
# ans = features[tag]['feature']  #取得是哪些feature
# tag = train[ans]  #取得feature資料
#print(ans)
#ax.set_title("tag_0")
# for feature in ans:
#     check = train[[feature, 'resp']].groupby(['resp'], as_index=False).mean().sort_values(by='resp', ascending=False)
#     ft = pd.Series(train[feature])
#     ft.plot(lw=3)
    #print(check)
# plt.legend(loc="upper left");
# gc.collect();

In [8]:
# tag = features
# new_tag = []
# for tg in tag:
#     if (tag[tg][7] == True) and (tag[tg][8] == True):
#         new_tag.append(tg)
#         print(tg)
# for tg in new_tag:
#     view = tag[tg] == True
#     print('tag is ' + tg)
#     print(tag[view]['feature'])

In [9]:
# check_1 = x[['feature_7', 'feature_55']].groupby(['feature_7'], as_index=False).mean().sort_values(by='feature_55', ascending=False)
# check_2 = x[['feature_7', 'feature_120']].groupby(['feature_7'], as_index=False).mean().sort_values(by='feature_120', ascending=False)
# check_3 = x[['feature_7', 'feature_121']].groupby(['feature_7'], as_index=False).mean().sort_values(by='feature_121', ascending=False)

# print('This is Check_1')
# print(check_1)
# print('This is Check_2')
# print(check_2[check_2['feature_120'].isnull()])
# print(check_2['feature_120'].isnull().sum())
# print('This is Check_3')
# print(check_3)
# print(check_3['feature_121'].isnull().sum())

將分析資料以>0作為區分 判別此效果對哪幾些有顯著幫助
平均差為 0.0005447971881263638

In [10]:
%%time
num = 0
needft = []
i = 0
for feature in x:
    if (i != 7) and (i != 8 ):
        train[feature] = x[feature].apply(lambda x:1 if x > 0 else -1)
        check = train[[feature, 'resp']].groupby([feature], as_index=False).mean().sort_values(by='resp', ascending=False)
        if check['resp'][0] > check['resp'][1]:
#             num += check['resp'][0] - check['resp'][1]
            num = check['resp'][0] - check['resp'][1]
        else:
#             num += check['resp'][1] - check['resp'][0]
            num = check['resp'][1] - check['resp'][0]
        if (num - 0.0005447971881263638) > 0:    #差 - 平均差
            needft.append(feature)
#             print(feature)
#             x[feature] = x[feature].apply(lambda x:1 if x > 0 else -1)
#             print(feature)
        del check , num
    i+=1
del i

CPU times: user 3min 55s, sys: 1min 31s, total: 5min 27s
Wall time: 5min 30s


將有顯著幫助的數值改變

In [11]:
%%time
for feature in needft:
    x[feature] = x[feature].apply(lambda x:1 if x > 0 else -1)

CPU times: user 1min 17s, sys: 18.3 s, total: 1min 35s
Wall time: 1min 36s


In [12]:
clf = xgb.XGBClassifier(       #XGB參數 參考NOTEBOOK
    n_estimators=500,
    max_depth=11,
    min_child_weight=9.15,
    gamma=0.59,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.7,
    alpha=10.4,
    nthread=5,
    missing=-999,
    random_state=2020,
    tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)

In [13]:
%%time
clf.fit(x,y)

CPU times: user 1min 15s, sys: 4.09 s, total: 1min 19s
Wall time: 1min 20s


XGBClassifier(alpha=10.4, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.59, gpu_id=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=11,
              min_child_weight=9.15, missing=-999,
              monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
              n_estimators=500, n_jobs=5, nthread=5, num_parallel_tree=1,
              random_state=2020, reg_alpha=10.3999996, reg_lambda=1,
              scale_pos_weight=1, subsample=0.9, tree_method='gpu_hist',
              validate_parameters=1, verbosity=None)

In [14]:
%%time
for (test_df, sample_prediction_df) in iter_test:
    X_test = test_df.loc[:, test_df.columns.str.contains('feature')]    #取得test_df資料
    y_preds = clf.predict(X_test)
    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)

CPU times: user 3min 52s, sys: 442 ms, total: 3min 52s
Wall time: 3min 55s
